In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [21]:
# Back-Propagation Neural Networks
# 
# Written in Python.  See http://www.python.org/
# Placed in the public domain.
# Neil Schemenauer <nas@arctrix.com>

import math
import random
import string

random.seed(0)

# calculate a random number where:  a <= rand < b
def rand(a, b):
    return (b-a)*random.random() + a

# Make a matrix (we could use NumPy to speed this up)
def makeMatrix(I, J, fill=0.0):
    m = []
    for i in range(I):
        m.append([fill]*J)
    return m

# our sigmoid function, tanh is a little nicer than the standard 1/(1+e^-x)
def sigmoid(x):
    return math.tanh(x)

# derivative of our sigmoid function, in terms of the output (i.e. y)
def dsigmoid(y):
    return 1.0 - y**2

class NN:
    def __init__(self, ni, nh, no):
        # number of input, hidden, and output nodes
        self.ni = ni + 1 # +1 for bias node
        self.nh = nh
        self.no = no

        # activations for nodes
        self.ai = [1.0]*self.ni
        self.ah = [1.0]*self.nh
        self.ao = [1.0]*self.no

        # create weights
        self.wi = makeMatrix(self.ni, self.nh)
        self.wo = makeMatrix(self.nh, self.no)
        # set them to random vaules
        for i in range(self.ni):
            for j in range(self.nh):
                self.wi[i][j] = rand(-0.2, 0.2)
        for j in range(self.nh):
            for k in range(self.no):
                self.wo[j][k] = rand(-2.0, 2.0)

        # last change in weights for momentum   
        self.ci = makeMatrix(self.ni, self.nh)
        self.co = makeMatrix(self.nh, self.no)

    def update(self, inputs):
        if len(inputs) != self.ni-1:
            #raise ValueError, 'wrong number of inputs'
            raise ValueError
        # input activations
        for i in range(self.ni-1):
            #self.ai[i] = sigmoid(inputs[i])
            self.ai[i] = inputs[i]

        # hidden activations
        for j in range(self.nh):
            summ = 0.0
            for i in range(self.ni):
                summ = summ + self.ai[i] * self.wi[i][j]
            self.ah[j] = sigmoid(summ)

        # output activations
        for k in range(self.no):
            summ = 0.0
            for j in range(self.nh):
                summ = summ + self.ah[j] * self.wo[j][k]
            self.ao[k] = sigmoid(summ)

        return self.ao[:]


    def backPropagate(self, targets, N, M):
        if len(targets) != self.no:
            #raise ValueError, 'wrong number of target values'
            raise ValueError
        # calculate error terms for output
        output_deltas = [0.0] * self.no
        for k in range(self.no):
            error = targets[k]-self.ao[k]
            output_deltas[k] = dsigmoid(self.ao[k]) * error

        # calculate error terms for hidden
        hidden_deltas = [0.0] * self.nh
        for j in range(self.nh):
            error = 0.0
            for k in range(self.no):
                error = error + output_deltas[k]*self.wo[j][k]
            hidden_deltas[j] = dsigmoid(self.ah[j]) * error

        # update output weights
        for j in range(self.nh):
            for k in range(self.no):
                change = output_deltas[k]*self.ah[j]
                self.wo[j][k] = self.wo[j][k] + N*change + M*self.co[j][k]
                self.co[j][k] = change
                #print N*change, M*self.co[j][k]

        # update input weights
        for i in range(self.ni):
            for j in range(self.nh):
                change = hidden_deltas[j]*self.ai[i]
                self.wi[i][j] = self.wi[i][j] + N*change + M*self.ci[i][j]
                self.ci[i][j] = change

        # calculate error
        error = 0.0
        for k in range(len(targets)):
            error = error + 0.5*(targets[k]-self.ao[k])**2
        return error


    def test(self, patterns):
        for p in patterns:
            print (p[0], '->', self.update(p[0]))

    def weights(self):
        print ('Input weights:')
        for i in range(self.ni):
            print (self.wi[i])
        print
        print ('Output weights:')
        for j in range(self.nh):
            print (self.wo[j])

    def train(self, patterns, iterations=1000, N=0.5, M=0.1):
        # N: learning rate
        # M: momentum factor
        for i in range(iterations):
            error = 0.0
            for p in patterns:
                inputs = p[0]
                targets = p[1]
                self.update(inputs)
                error = error + self.backPropagate(targets, N, M)
                print("Inputs  ",inputs)
                print("Targets ",targets)
                print("Error   ",error)
            if i % 100 == 0:
                pass #print 'error %-14f' % error


def demo():
    # Teach network XOR function
    pat = [
        [[0,0], [1]],
        [[0,1], [0]],
        [[1,0], [0]],
        [[1,1], [1]]
    ]

    # create a network with two input, two hidden, and one output nodes
    n = NN(2, 2, 1)
    # train it with some patterns
    n.train(pat)
    # test it
    n.test(pat)



if __name__ == '__main__':
    demo()

Inputs   [0, 0]
Targets  [1]
Error    0.4656047471673693
Inputs   [0, 1]
Targets  [0]
Error    0.730226344900226
Inputs   [1, 0]
Targets  [0]
Error    0.9516514595462229
Inputs   [1, 1]
Targets  [1]
Error    1.3411376382817872
Inputs   [0, 0]
Targets  [1]
Error    0.0404586921784082
Inputs   [0, 1]
Targets  [0]
Error    0.4139938131842316
Inputs   [1, 0]
Targets  [0]
Error    0.7500782543716128
Inputs   [1, 1]
Targets  [1]
Error    0.7726038620576039
Inputs   [0, 0]
Targets  [1]
Error    0.060767468093509656
Inputs   [0, 1]
Targets  [0]
Error    0.37886420230128404
Inputs   [1, 0]
Targets  [0]
Error    0.6390658541635375
Inputs   [1, 1]
Targets  [1]
Error    0.7042372481167746
Inputs   [0, 0]
Targets  [1]
Error    0.10253286792983439
Inputs   [0, 1]
Targets  [0]
Error    0.37347453138390474
Inputs   [1, 0]
Targets  [0]
Error    0.577956817787525
Inputs   [1, 1]
Targets  [1]
Error    0.6910460808513846
Inputs   [0, 0]
Targets  [1]
Error    0.1269565830129054
Inputs   [0, 1]
Targets  [0]

Error    0.020736952453808792
Inputs   [1, 0]
Targets  [0]
Error    0.03690472249329163
Inputs   [1, 1]
Targets  [1]
Error    0.04243867986296694
Inputs   [0, 0]
Targets  [1]
Error    0.005930455176314171
Inputs   [0, 1]
Targets  [0]
Error    0.020231244271239086
Inputs   [1, 0]
Targets  [0]
Error    0.03600435680864485
Inputs   [1, 1]
Targets  [1]
Error    0.041380867635216534
Inputs   [0, 0]
Targets  [1]
Error    0.005758372973840113
Inputs   [0, 1]
Targets  [0]
Error    0.019748333314060666
Inputs   [1, 0]
Targets  [0]
Error    0.03514500909080228
Inputs   [1, 1]
Targets  [1]
Error    0.04037192076432462
Inputs   [0, 0]
Targets  [1]
Error    0.005595053104017106
Inputs   [0, 1]
Targets  [0]
Error    0.019286754946046484
Inputs   [1, 0]
Targets  [0]
Error    0.03432399224274785
Inputs   [1, 1]
Targets  [1]
Error    0.0394086051664681
Inputs   [0, 0]
Targets  [1]
Error    0.005439871901899953
Inputs   [0, 1]
Targets  [0]
Error    0.01884516454600237
Inputs   [1, 0]
Targets  [0]
Error 

Targets  [0]
Error    0.010808304003667647
Inputs   [1, 1]
Targets  [1]
Error    0.012149703100451803
Inputs   [0, 0]
Targets  [1]
Error    0.0014178169348482204
Inputs   [0, 1]
Targets  [0]
Error    0.005985417675185953
Inputs   [1, 0]
Targets  [0]
Error    0.01072180688251654
Inputs   [1, 1]
Targets  [1]
Error    0.012051007773899818
Inputs   [0, 0]
Targets  [1]
Error    0.0014048910254276957
Inputs   [0, 1]
Targets  [0]
Error    0.00593752091412673
Inputs   [1, 0]
Targets  [0]
Error    0.010636636679463796
Inputs   [1, 1]
Targets  [1]
Error    0.011953842101363127
Inputs   [0, 0]
Targets  [1]
Error    0.0013921809062638528
Inputs   [0, 1]
Targets  [0]
Error    0.005890356218463569
Inputs   [1, 0]
Targets  [0]
Error    0.010552763529493385
Inputs   [1, 1]
Targets  [1]
Error    0.011858171375465063
Inputs   [0, 0]
Targets  [1]
Error    0.0013796813942902368
Inputs   [0, 1]
Targets  [0]
Error    0.005843907200094479
Inputs   [1, 0]
Targets  [0]
Error    0.010470158449995694
Inputs   [1

Error    0.0007497733718482091
Inputs   [0, 1]
Targets  [0]
Error    0.0034034009941119707
Inputs   [1, 0]
Targets  [0]
Error    0.006121854262379608
Inputs   [1, 1]
Targets  [1]
Error    0.006827961381994087
Inputs   [0, 0]
Targets  [1]
Error    0.0007457924232010339
Inputs   [0, 1]
Targets  [0]
Error    0.0033872311211265945
Inputs   [1, 0]
Targets  [0]
Error    0.006092977891027409
Inputs   [1, 1]
Targets  [1]
Error    0.0067953531709508365
Inputs   [0, 0]
Targets  [1]
Error    0.0007418506702218374
Inputs   [0, 1]
Targets  [0]
Error    0.003371209165419274
Inputs   [1, 0]
Targets  [0]
Error    0.006064364601800096
Inputs   [1, 1]
Targets  [1]
Error    0.006763044574643529
Inputs   [0, 0]
Targets  [1]
Error    0.0007379475536723612
Inputs   [0, 1]
Targets  [0]
Error    0.003355333135140101
Inputs   [1, 0]
Targets  [0]
Error    0.0060360108616325766
Inputs   [1, 1]
Targets  [1]
Error    0.006731031542164952
Inputs   [0, 0]
Targets  [1]
Error    0.0007340825247266255
Inputs   [0, 1]
T

Error    0.0005075437183882992
Inputs   [0, 1]
Targets  [0]
Error    0.0023963872108420624
Inputs   [1, 0]
Targets  [0]
Error    0.0043212086364358485
Inputs   [1, 1]
Targets  [1]
Error    0.004799865370970313
Inputs   [0, 0]
Targets  [1]
Error    0.0005056202091967706
Inputs   [0, 1]
Targets  [0]
Error    0.0023881800408249963
Inputs   [1, 0]
Targets  [0]
Error    0.004306511760163602
Inputs   [1, 1]
Targets  [1]
Error    0.004783359372530969
Inputs   [0, 0]
Targets  [1]
Error    0.0005037103069390464
Inputs   [0, 1]
Targets  [0]
Error    0.0023800272124308687
Inputs   [1, 0]
Targets  [0]
Error    0.004291911804945222
Inputs   [1, 1]
Targets  [1]
Error    0.004766963048606525
Inputs   [0, 0]
Targets  [1]
Error    0.0005018138716289735
Inputs   [0, 1]
Targets  [0]
Error    0.0023719281947221747
Inputs   [1, 0]
Targets  [0]
Error    0.004277407826486469
Inputs   [1, 1]
Targets  [1]
Error    0.004750675324233886
Inputs   [0, 0]
Targets  [1]
Error    0.0004999307651645601
Inputs   [0, 1]


...

Targets  [1]
Error    0.0026175112032178305
Inputs   [0, 0]
Targets  [1]
Error    0.00026072758293208627
Inputs   [0, 1]
Targets  [0]
Error    0.0013069485623016502
Inputs   [1, 0]
Targets  [0]
Error    0.002366244684248335
Inputs   [1, 1]
Targets  [1]
Error    0.002612422886134482
Inputs   [0, 0]
Targets  [1]
Error    0.0002601770688071933
Inputs   [0, 1]
Targets  [0]
Error    0.0013044194405046486
Inputs   [1, 0]
Targets  [0]
Error    0.002361694823453489
Inputs   [1, 1]
Targets  [1]
Error    0.00260735379841743
Inputs   [0, 0]
Targets  [1]
Error    0.0002596287452145858
Inputs   [0, 1]
Targets  [0]
Error    0.0013018998366836662
Inputs   [1, 0]
Targets  [0]
Error    0.0023571620210714097
Inputs   [1, 1]
Targets  [1]
Error    0.00260230383250523
Inputs   [0, 0]
Targets  [1]
Error    0.0002590825994142945
Inputs   [0, 1]
Targets  [0]
Error    0.0012993896978084538
Inputs   [1, 0]
Targets  [0]
Error    0.002352646182316976
Inputs   [1, 1]
Targets  [1]
Error    0.0025972728816317956
Inp

Error    0.00021671022044618616
Inputs   [0, 1]
Targets  [0]
Error    0.001102905407841732
Inputs   [1, 0]
Targets  [0]
Error    0.0019989555461291625
Inputs   [1, 1]
Targets  [1]
Error    0.0022036099088883826
Inputs   [0, 0]
Targets  [1]
Error    0.00021632196301845375
Inputs   [0, 1]
Targets  [0]
Error    0.001101088307743213
Inputs   [1, 0]
Targets  [0]
Error    0.0019956825818135022
Inputs   [1, 1]
Targets  [1]
Error    0.0021999706112850975
Inputs   [0, 0]
Targets  [1]
Error    0.00021593501952484406
Inputs   [0, 1]
Targets  [0]
Error    0.0010992770380574483
Inputs   [1, 0]
Targets  [0]
Error    0.001992420080728336
Inputs   [1, 1]
Targets  [1]
Error    0.002196343015485015
Inputs   [0, 0]
Targets  [1]
Error    0.00021554938345832718
Inputs   [0, 1]
Targets  [0]
Error    0.0010974715710664283
Inputs   [1, 0]
Targets  [0]
Error    0.001989167993263177
Inputs   [1, 1]
Targets  [1]
Error    0.00219272706575131
Inputs   [0, 0]
Targets  [1]
Error    0.0002151650483542079
Inputs   [0,

Inputs   [1, 1]
Targets  [1]
Error    0.0019368224951149002
Inputs   [0, 0]
Targets  [1]
Error    0.00018812930840397747
Inputs   [0, 1]
Targets  [0]
Error    0.0009682516815586607
Inputs   [1, 0]
Targets  [0]
Error    0.0017563083216030086
Inputs   [1, 1]
Targets  [1]
Error    0.0019339926438991107
Inputs   [0, 0]
Targets  [1]
Error    0.000187832202493476
Inputs   [0, 1]
Targets  [0]
Error    0.0009668420374352397
Inputs   [1, 0]
Targets  [0]
Error    0.0017537669205147481
Inputs   [1, 1]
Targets  [1]
Error    0.0019311708510567126
Inputs   [0, 0]
Targets  [1]
Error    0.00018753598438866868
Inputs   [0, 1]
Targets  [0]
Error    0.0009654363942035926
Inputs   [1, 0]
Targets  [0]
Error    0.0017512327065449754
Inputs   [1, 1]
Targets  [1]
Error    0.0019283570825760592
Inputs   [0, 0]
Targets  [1]
Error    0.0001872406502039864
Inputs   [0, 1]
Targets  [0]
Error    0.0009640347350297132
Inputs   [1, 0]
Targets  [0]
Error    0.0017487056495324275
Inputs   [1, 1]
Targets  [1]
Error    0

Targets  [0]
Error    0.0015632734029476648
Inputs   [1, 1]
Targets  [1]
Error    0.001719790001338187
Inputs   [0, 0]
Targets  [1]
Error    0.00016546798188423457
Inputs   [0, 1]
Targets  [0]
Error    0.0008601015178818202
Inputs   [1, 0]
Targets  [0]
Error    0.001561249720062519
Inputs   [1, 1]
Targets  [1]
Error    0.0017175458352173246
Inputs   [0, 0]
Targets  [1]
Error    0.0001652350168577948
Inputs   [0, 1]
Targets  [0]
Error    0.0008589828039082833
Inputs   [1, 0]
Targets  [0]
Error    0.0015592311613226958
Inputs   [1, 1]
Targets  [1]
Error    0.0017153073818594595
Inputs   [0, 0]
Targets  [1]
Error    0.00016500267336243087
Inputs   [0, 1]
Targets  [0]
Error    0.0008578669291705611
Inputs   [1, 0]
Targets  [0]
Error    0.0015572177074661007
Inputs   [1, 1]
Targets  [1]
Error    0.0017130746196996718
Inputs   [0, 0]
Targets  [1]
Error    0.0001647709489676853
Inputs   [0, 1]
Targets  [0]
Error    0.0008567538829813405
Inputs   [1, 0]
Targets  [0]
Error    0.0015552093393264

Error    0.00152942496578905
Inputs   [0, 0]
Targets  [1]
Error    0.00014581068488720935
Inputs   [0, 1]
Targets  [0]
Error    0.0007651739470115513
Inputs   [1, 0]
Targets  [0]
Error    0.0013898997663621914
Inputs   [1, 1]
Targets  [1]
Error    0.0015276402641838148
Inputs   [0, 0]
Targets  [1]
Error    0.000145627421060738
Inputs   [0, 1]
Targets  [0]
Error    0.0007642836884430386
Inputs   [1, 0]
Targets  [0]
Error    0.0013882921348737866
Inputs   [1, 1]
Targets  [1]
Error    0.0015258596288211378
Inputs   [0, 0]
Targets  [1]
Error    0.0001454445943233109
Inputs   [0, 1]
Targets  [0]
Error    0.0007633954526627355
Inputs   [1, 0]
Targets  [0]
Error    0.0013866881434595763
Inputs   [1, 1]
Targets  [1]
Error    0.0015240830459564189
Inputs   [0, 0]
Targets  [1]
Error    0.0001452622031458779
Inputs   [0, 1]
Targets  [0]
Error    0.0007625092328508633
Inputs   [1, 0]
Targets  [0]
Error    0.0013850877798782994
Inputs   [1, 1]
Targets  [1]
Error    0.001522310501906518
Inputs   [0,

In [53]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
model = keras.Sequential()
input_layer = keras.layers.Dense(8,input_dim=2,activation='tanh')
model.add(input_layer)
hidden_layer = keras.layers.Dense(3,activation='sigmoid')
model.add(hidden_layer)
output_layer = keras.layers.Dense(1,activation='sigmoid')
model.add(output_layer)

sgd = keras.optimizers.SGD(lr=1)
model.compile(optimizer=sgd,loss='binary_crossentropy')

x = np.array([[0,0],[0,1],[1,0],[1,1]])
y = np.array([[1],[0],[0],[1]])
for i in range (1,50):
    model.fit(x,y,epochs=10,steps_per_epoch=50)
    results = model.predict(x,verbose=0,steps=1)
    print(results)
print("Hasil akhir =")    
print(results)    

Epoch 1/10
40/50 [=======================>......] - 0s 795us/step - loss: 0.9244
[[0.41959164]
 [0.419757  ]
 [0.41490245]
 [0.41850856]]
Epoch 1/10
40/50 [=======================>......] - 0s 789us/step - loss: 0.8434
[[0.49380735]
 [0.47718886]
 [0.4715928 ]
 [0.46997735]]
Epoch 1/10
40/50 [=======================>......] - 0s 781us/step - loss: 0.8202
[[0.5977059]
 [0.4865394]
 [0.479768 ]
 [0.4694099]]
Epoch 1/10
40/50 [=======================>......] - 0s 788us/step - loss: 0.7073
[[0.8855512 ]
 [0.42788738]
 [0.42327613]
 [0.4170485 ]]
Epoch 1/10
40/50 [=======================>......] - 0s 788us/step - loss: 0.6014
[[0.94920516]
 [0.43141684]
 [0.41989827]
 [0.43463624]]
Epoch 1/10
40/50 [=======================>......] - 0s 793us/step - loss: 0.5709
[[0.9411653 ]
 [0.30758047]
 [0.24274647]
 [0.35836208]]
Epoch 1/10
40/50 [=======================>......] - 0s 778us/step - loss: 0.4993
[[0.92170167]
 [0.12580433]
 [0.10824636]
 [0.37405595]]
Epoch 1/10
40/50 [====================

In [50]:
model.fit(x,y,epochs=10,steps_per_epoch=50)

results = model.predict(x,verbose=0,steps=1)
print(results)

Epoch 1/10
40/50 [=======================>......] - 0s 800us/step - loss: 0.0085
[[0.98937047]
 [0.00825021]
 [0.00816321]
 [0.9943683 ]]


AttributeError: module 'tensorflow' has no attribute 'placeholder'